In [ ]:
from config3 import *

As it is not possible to only predict new data using a prefitted hdbscan model, Ignacio adised to compute the distance of the song features with all other songs in the database and to look for the cluster of the closest song. This seems not to make sense to me, I could directly choose the five closest neghbours and not use the models. 
There is the same problem with the TSNE which is the dimensionality reduction technique that was the most satisfactory on my dataset and that I used. 

To conclude I decided to use the kmeans technique, because it is the only one that is somehow satisfactory (it won't give me errors with minus ones)and that allows to effectively predict cluster of the song chosen by the user using the model. 

In [394]:
def Vynila():
    import pandas as pd
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    #Initialize SpotiPy with user credentias #
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))
    import pickle
    from sklearn.preprocessing import StandardScaler

    #getting the input from the user 
    print("input the title of the song")
    title=input() 
    print("input the name of the artist or group")
    artist=input() 

    #getting the input of the user in the right format
    title=title.title()
    artist=artist.title()

    #fecthing five ids for the user input
    list_of_ids = []
    list_of_titles = []
    list_of_artists = []
    songs=pd.DataFrame(columns=['title', 'artist'])
    limit=5
    results = sp.search(q="tracks: "+title+" artist: "+artist, limit=limit)
    
    #building a dataframe with the results of the fetch
    for i in range (0, limit):
        id = results['tracks']['items'][i]['id'] 
        list_of_ids.append(id)
        title = results['tracks']['items'][i]['name'] 
        list_of_titles.append(title)
        artist = results['tracks']['items'][i]['artists'][0]['name']
        list_of_artists.append(artist) 
    songs["title"]=list_of_titles
    songs["artist"]=list_of_artists
    songs["id"]=list_of_ids

    #get the user to choose which song he typed
    display(songs)
    print("choose the row (from 0 to 4) of the song that corresponds to your search")
    X=input()

    #select the song the user chose
    if X=="0":
        id_real=songs.iloc[0,2] 
        selected_song=pd.DataFrame(songs.iloc[0,:])
    elif X=="1":
        id_real=songs.iloc[1,2]
        selected_song=pd.DataFrame(songs.iloc[1,:])
    elif X=="2":
        id_real=songs.iloc[2,2]
        selected_song=pd.DataFrame(songs.iloc[2,:])
    elif X=="3":
        id_real=songs.iloc[3,2]
        selected_song=pd.DataFrame(songs.iloc[3,:])
    elif X=="4":
        id_real=songs.iloc[4,2]
        selected_song=pd.DataFrame(songs.iloc[4,:])
    selected_song=selected_song.transpose()
    
    #fetch audio features from spotify and merge with song id and title 
    dict_1 = sp.audio_features(id_real)[0] 
    feature=pd.DataFrame(dict_1, index=[0])
    merged = selected_song.merge(feature, how="left", left_on = "id", right_on = "id").drop_duplicates().reset_index()

    #keeping only relevant columns
    data=merged[["danceability", "energy", "speechiness",  "acousticness", "instrumentalness", "valence"]]

    
    #scale the data
    with open("data_scaler.pickle", "rb") as f:
            scaler = pickle.load(f)

    data_scaled=scaler.transform(data)

    data_scaled_df = pd.DataFrame(data_scaled, columns = data.columns)

    #predict the cluster of the data
    with open("kmeans_4.pickle", "rb") as file:
                model = pickle.load(file)

    cluster=model.predict(data_scaled)[0]

    #lodaing both hot song data frame and all_songs dataframe
    hot_songs = pd.read_csv('../../Day_2/Afternoon/hot_songs_with_features.csv', sep=";") 
    all_songs=pd.read_csv('all_songs_cluster.csv', sep=";") 
    
    #checking if the song is contained in the hot songs, selecting five songs from the right cluster and showing it to the user
    Y=hot_songs["id"].str.contains(merged.iloc[0,3]).tolist()
    Y = [x for x in Y if str(x) != 'nan']
    if (any(Y)==True):
        all_songs_selection = all_songs[(all_songs["cluster_kmeans"]==cluster)&(all_songs["database"]=="hot_songs")]
        display(all_songs_selection[["song", "artist"]].sample(n=5, axis=0))
    else : 
        all_songs_selection = all_songs[all_songs["cluster_kmeans"]==cluster]
        display(all_songs_selection[["song", "artist"]].sample(n=5, axis=0))